In [39]:
from pathlib import Path
import arxiv
from typing import Union, List, Dict

PATH_ALL_PAPER_TRAIN: str = "../sota/dataset/train"
PATH_ALL_PAPER_VALIDATION: str = "../sota/dataset/validation"

all_paper_id_iter = Path(PATH_ALL_PAPER_TRAIN).glob("*")
all_paper_dict: dict = {}

cnt = 400
for paper_src in all_paper_id_iter:
  # print(f"{paper_src.name = }")
  all_paper_dict[paper_src.name] = {"name": None, "title": None}
  all_paper_dict[paper_src.name]["name"] = paper_src.name
  cnt -= 1
  if cnt == 0:
    break

# all_paper
all_paper_dict
len(all_paper_dict.keys())

400

In [40]:
def checkAndCreateFolder(dirpath: str,
                         filename: str=None, 
                         basepath: str="sources",
                         demo: bool=False
                        ) -> bool:
  '''
  return `True` if the file exists.
  '''
  base_path = Path(basepath)
  folder_path = Path(basepath).joinpath(dirpath)
  file_path = folder_path.joinpath(filename)
  if not base_path.exists():
    base_path.mkdir()
  if not folder_path.exists():
    # print(f"['{folder_path}'] doesn't exist. Creating one for you...")
    print(f"['{folder_path}'] doesn't exist. ", end="" if not demo else "\n")
    if not demo:
      print("Creating one for you...")
      folder_path.mkdir()
  else:
    if demo:
      print(f"['{folder_path}'] does exist!")
  
  return file_path.exists()

# checkAndCreateFolder(dirpath=paper_src.name.split(".")[0], filename=paper_src.name, demo=True)
_paper_src_name = "2005.05005v2"
# _paper_src_name = "2309.15112v4"
checkAndCreateFolder(dirpath=_paper_src_name.split(".")[0], filename=_paper_src_name, demo=True)

['sources/2005'] does exist!


True

In [41]:
flag: int = 0
step: int = 400
probe: slice = slice(flag, flag + step)
all_paper_key_list = list(all_paper_dict.keys())[probe]
search_by_id = arxiv.Search(id_list=all_paper_key_list)
# Skip some bad sources
# search_by_id.id_list.remove("2309.15112v4")

In [69]:
client = arxiv.Client(
  page_size=1000,
  delay_seconds=3,
  num_retries=3,
)
from tqdm.auto import tqdm, trange

pbar = tqdm(client.results(search_by_id), 
            total=probe.stop - probe.start,
            miniters=1,
            mininterval=0,
            desc="Starting process...")
skipped_list = []
for paper in pbar: 
  paper_id = paper.get_short_id()
  all_paper_dict[paper_id] = {}
  all_paper_dict[paper_id]["name"] = paper_id
  all_paper_dict[paper_id]["title"] = paper.title
  pbar.set_description(f"Processing {paper_id}")
  if not checkAndCreateFolder(dirpath=paper_id.split(".")[0], filename=paper_id):
    # Skip the task if the file already exist.
    pbar.write(f"{pbar.n+1}: [{paper_id}] {paper.title}")
    paper.download_source(dirpath="./sources/" + paper_id.split(".")[0], filename=paper_id)
  else:
    skipped_list.append(paper_id)

print(f"Number of skipped data: {len(skipped_list)}")


Starting process...:   0%|          | 0/400 [00:00<?, ?it/s]

* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped
* Skipped


In [66]:
# all_paper_dict
pbar.n

400

In [50]:
import pandas as pd
info_table = pd.DataFrame(all_paper_dict).T
info_table

,name,title
2210.05954v2,2210.05954v2,Image Projective Transformation Rectification ...
2203.16804v1,2203.16804v1,BRIO: Bringing Order to Abstractive Summarization
2204.00943v4,2204.00943v4,Efficient Convolutional Neural Networks on Ras...
1803.09074v1,1803.09074v1,Multi-range Reasoning for Machine Comprehension
2212.05853v3,2212.05853v3,DeepCut: Unsupervised Segmentation using Graph...
...,...,...
0803.4030v1,0803.4030v1,Learning Sequences
1104.4203v1,1104.4203v1,Pattern matching in Lempel-Ziv compressed stri...
1504.08259v4,1504.08259v4,Edit Distance for Pushdown Automata
1401.2974v1,1401.2974v1,Software Tool Combining Fault Masking with Use...


In [51]:
# info_table.value_counts()
info_table.count()

name     520
title    400
dtype: int64

In [54]:
# _fmt: str = "json"
_fmt: str = "csv"
info_table.to_csv(f"record_{probe.start}-{probe.stop}.{_fmt}")